# TRATAMIENTO DE DATOS CON PYSPARK

In [9]:
# 1. CONFIGURACIÓN E IMPORTACIONES

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline



# Iniciamos Spark
spark = SparkSession.builder \
    .appName("FinPlus_ETL_Limpieza") \
    .getOrCreate()

# Rutas de los datos
DATA_PATH = "/home/jovyan/work/data/"  
OUTPUT_PATH = "/home/jovyan/work/data/curated/"

In [10]:
# 2. INGESTA DE DATOS 

# Cargar CLIENTS.csv
# Usamos header=True para leer la cabecera e inferSchema=True para detectar números
df_clients = spark.read.csv(DATA_PATH + "CLIENTS.csv", header=True, inferSchema=True, sep=',')

# Cargar BEHAVIOURAL.parquet
df_behav = spark.read.parquet(DATA_PATH + "BEHAVIOURAL.parquet")

# Verificamos qué columnas tenemos 
print("Esquema Clients:")
df_clients.printSchema()

print("Esquema Behavioural:")
df_behav.printSchema()

Esquema Clients:
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = true)
 |-- EDUCATION: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = true)
 |-- HOME_SENIORITY: double (nullable = true)
 |-- LAST_UPDATE: double (nullable = true)
 |-- OWN_INSURANCE_CAR: string (nullable = true)
 |-- CAR_AGE: double (nullable = true)
 |-- FAMILY_SIZE: double (nullable = true)
 |-- REACTIVE_SCORING: double (nullable = true)
 |-- PROACTIVE_SCORING: double (nullable = true)
 |-- BEHAVIORAL_SCORING: double (nullable = true)
 |-- DAYS_LAST_INFO_CHANGE: d

Primero analizamos las variables de tipo string. Hay de distintos tipos:

- Categóricas puras: "NAME_PRODUCT_TYPE", "GENDER", "EDUCATION", "MARITAL_STATUS", 
    "HOME_SITUATION", "OWN_INSURANCE_CAR", "OCCUPATION", 
    "HOME_OWNER", "EMPLOYER_ORGANIZATION_TYPE", "CURRENCY" (en ambas tablas).

- Categóricas numéricas: "NON_COMPLIANT_CONTRACT", "DIGITAL_CLIENT".

- Fecha: "DATE" (sólo en behavioural).

- Identificadores: "CLIENT_ID" (en ambas tablas), "CONTRACT_ID".

Analizamos las filas duplicadas

In [11]:
def auditar_duplicados_completo(df, nombre_tabla):
    print(f"VERIFICACIÓN DE DUPLICADOS: {nombre_tabla}\n ")
    
    # 1. Cálculos Básicos
    total_rows = df.count()
    distinct_rows = df.distinct().count()
    num_duplicados = total_rows - distinct_rows
    
    print(f"• Total filas:      {total_rows}")
    print(f"• Filas únicas:     {distinct_rows}")
    print(f"• Duplicados:       {num_duplicados}")
    
    # 2. Lógica Condicional
    if num_duplicados > 0:
        pct = (num_duplicados / total_rows) * 100
        print(f"\n AVISO: Hay {num_duplicados} filas repetidas ({pct:.2f}%).")
        print("   Mostrando ejemplos de filas idénticas:")
        
        # Esta parte solo se ejecuta si hay duplicados 
        # Agrupamos por TODAS las columnas para encontrar filas 100% idénticas
        (df.groupBy(df.columns)
           .count()
           .where(F.col("count") > 1)
           .orderBy(F.col("count").desc()) # Ponemos las más repetidas arriba
           .show(5, truncate=False))
        
    else:
        print("\nLimpio. No existen filas duplicadas exactas.")
        

# --- EJECUCIÓN ---
auditar_duplicados_completo(df_clients, "CLIENTS")
auditar_duplicados_completo(df_behav, "BEHAVIOURAL")

VERIFICACIÓN DE DUPLICADOS: CLIENTS
 
• Total filas:      162977
• Filas únicas:     162977
• Duplicados:       0

Limpio. No existen filas duplicadas exactas.
VERIFICACIÓN DE DUPLICADOS: BEHAVIOURAL
 
• Total filas:      1724854
• Filas únicas:     1724854
• Duplicados:       0

Limpio. No existen filas duplicadas exactas.


Ahora analizamos los posibles IDs duplicados

In [12]:
def auditar_clave_primaria(df, col_id, nombre_tabla):
    print(f"VERIFICACIÓN DE CLAVE ÚNICA ({col_id}): {nombre_tabla}\n ")
    
    # 1. Contamos filas totales
    total = df.count()
    
    # 2. Contamos IDs únicos
    unicos = df.select(col_id).distinct().count()
    
    # 3. Diferencia
    dif = total - unicos
    
    if dif > 0:
        print(f"AVISO: Hay {dif} IDs repetidos que NO son filas idénticas.")
        print("   Esto significa que tienes clientes con datos conflictivos.")
        print("   Ejemplo de IDs repetidos:")
        
        # Mostramos cuáles son los culpables
        (df.groupBy(col_id)
           .count()
           .where(F.col("count") > 1)
           .show(5))
    else:
        print(f"CORRECTO: La columna {col_id} es una clave primaria única.")

# Ejecutamos solo para CLIENTS
auditar_clave_primaria(df_clients, "CLIENT_ID", "CLIENTS")

VERIFICACIÓN DE CLAVE ÚNICA (CLIENT_ID): CLIENTS
 
CORRECTO: La columna CLIENT_ID es una clave primaria única.


Como vemos que no hay duplicados, no es necesario hacer limpieza de estos. Nos enfocaremos en los NaNs.

In [13]:
# 1. Definimos la función mejorada (le añadimos un título para que quede claro)
def inspeccionar_datos(df, nombre_tabla):
    
    print(f"INSPECCIÓN DE: {nombre_tabla}\n")
    
    print(f"Dimensiones: {df.count()} filas x {len(df.columns)} columnas")
    
    print(f"\n--- 1. Conteo de Nulos ---")
    # Calculamos nulos
    exprs_nulos = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]
    # Mostramos verticalmente para leer mejor
    df.agg(*exprs_nulos).show(vertical=True, truncate=False)

# 2. Ejecutamos la inspección 
inspeccionar_datos(df_clients, "CLIENTS")
inspeccionar_datos(df_behav, "BEHAVIOURAL")

INSPECCIÓN DE: CLIENTS

Dimensiones: 162977 filas x 45 columnas

--- 1. Conteo de Nulos ---
-RECORD 0-----------------------------
 CLIENT_ID                   | 0      
 NON_COMPLIANT_CONTRACT      | 0      
 NAME_PRODUCT_TYPE           | 0      
 GENDER                      | 0      
 TOTAL_INCOME                | 0      
 AMOUNT_PRODUCT              | 0      
 INSTALLMENT                 | 7      
 EDUCATION                   | 39640  
 MARITAL_STATUS              | 2      
 HOME_SITUATION              | 0      
 REGION_SCORE                | 0      
 AGE_IN_YEARS                | 0      
 JOB_SENIORITY               | 29174  
 HOME_SENIORITY              | 0      
 LAST_UPDATE                 | 0      
 OWN_INSURANCE_CAR           | 0      
 CAR_AGE                     | 107550 
 FAMILY_SIZE                 | 2      
 REACTIVE_SCORING            | 91901  
 PROACTIVE_SCORING           | 337    
 BEHAVIORAL_SCORING          | 32246  
 DAYS_LAST_INFO_CHANGE       | 1      
 NUMBER_OF_

Se observa que solamente tendremos que tratar nulls de la tabla de clientes. Pero además, no nos interesa eliminar filas porque eliminaríamos clientes, y menos en variables con tantos nulls como CAR_AGE, donde estaríamos eliminando más del 66% de los clientes. La estrategia ganadora en Big Data es "Imputar lo masivo, borrar lo anecdótico".

Primero de todo, eliminaremos a los clientes de los que no se tengan casi datos, pues prácticamente es como si no existiesen. Comprobamos primero si, aunque no tengamos info personal tienen movimientos.

In [14]:
# 1. Cargamos de nuevo CLIENTS original (solo para esta prueba)
df_raw = spark.read.csv(DATA_PATH + "CLIENTS.csv", header=True, inferSchema=True, sep=',')

# 2. Identificamos a los que vamos a borrar (los que tienen muchos nulos)
# thresh=25 mantenía a los buenos. Así que buscamos lo contrario.
# Para replicar la lógica inversa exacta, calculamos cuántos nulos tienen.

# Contamos cuántas columnas NO son nulas por fila
from itertools import chain
cols_check = df_raw.columns
expr = sum([F.when(F.col(c).isNotNull(), 1).otherwise(0) for c in cols_check])

# Filtramos los "Malos" (tienen menos de 25 columnas con datos)
df_zombies = df_raw.withColumn("non_nulls", expr).filter(F.col("non_nulls") < 25)

print(f"Detectados {df_zombies.count()} clientes 'Zombie' candidatos a borrar.")

# 3. CRUCE DE LA VERDAD
# Cruzamos estos Zombies con Behavioural. 
# Si sale 0, tu decisión fue perfecta. Si sale algo, cuidado.
zombies_con_dinero = df_zombies.join(df_behav, on="CLIENT_ID", how="inner")

count_risk = zombies_con_dinero.count()

if count_risk == 0:
    print(f"\n Ninguno de los clientes eliminados tenía actividad bancaria.")
else:
    print(f"\nAVISO: Hay {count_risk} clientes con pocos datos pero con movimientos.")

Detectados 1899 clientes 'Zombie' candidatos a borrar.

AVISO: Hay 6863 clientes con pocos datos pero con movimientos.


Con este resultado, esperaremos a eliminarlos después del JOIN de las tablas.

________________________________________________________________________________________________________________________

####### ELIMINAR "CLIENTES ZOMBIE" (Filas con demasiados nulos)

#######print(f"Filas antes de limpieza fina: {df_clients.count()}")

######## Tenemos unas 45 columnas. Si a un cliente le faltan más de 20 datos, no nos sirve.
######## thresh=25 significa: "Mantener solo si tiene al menos 25 columnas con datos válidos"
#######df_clients = df_clients.dropna(thresh=25) 

#######print(f"Filas tras limpieza fina: {df_clients.count()}")
________________________________________________________________________________________________________________________

In [15]:

# 3. LIMPIEZA Y TRANSFORMACIÓN

# A) LIMPIEZA DE STRINGS (CLIENTS)
# Lista de tus columnas categóricas reales (copiadas de tu esquema)
cols_categ_puras = [
    "NAME_PRODUCT_TYPE", "GENDER", "EDUCATION", "MARITAL_STATUS", 
    "HOME_SITUATION", "OWN_INSURANCE_CAR", "OCCUPATION", 
    "HOME_OWNER", "EMPLOYER_ORGANIZATION_TYPE", "CURRENCY"
]

# Normalizamos: quitamos espacios (trim) y pasamos a mayúsculas o minúsculas
for col_name in cols_categ_puras:
    # Solo si la columna existe en el dataframe
    if col_name in df_clients.columns:
        df_clients = df_clients.withColumn(col_name, F.trim(F.upper(F.col(col_name))))

# B) CONVERSIÓN DE FECHAS (BEHAVIOURAL)
# Tu columna DATE es string, hay que pasarla a formato fecha
# Spark suele ser listo, pero si falla, prueba con formato específico ej: "dd/MM/yyyy"
df_behav = df_behav.withColumn("DATE", F.to_date(F.col("DATE")))



In [16]:
# C) LIMPIEZA DE NULOS 

# 1. GRUPO "NO APLICA" (Rellenar con -1)
# Variables donde Nulo significa "No tiene" o "No disponible"
# CAR_AGE (66% nulos), JOB_SENIORITY, SCORING...
cols_flag = ["CAR_AGE", "JOB_SENIORITY", "REACTIVE_SCORING", "BEHAVIORAL_SCORING", "PROACTIVE_SCORING", "DAYS_LAST_INFO_CHANGE"]
df_clients = df_clients.fillna(-1, subset=cols_flag)

# 2. GRUPO "SIN HISTORIAL" (Rellenar con 0)
# El grupo de los 8770 nulos. Si no hay datos de préstamos, asumimos 0.
# Buscamos todas las columnas de préstamos (LOAN_) y estados (NUM_STATUS_)
cols_financieras = [c for c in df_clients.columns if c.startswith("LOAN_") or c.startswith("NUM_")]
# Añadimos otras que tengan sentido ser 0
cols_financieras.extend(["NUM_PREVIOUS_LOAN_APP", "NUMBER_OF_PRODUCTS", "Num_flag_insured"]) # Asegúrate de usar el nombre exacto (mayusc/minusc)

# Filtramos solo las que existen en el DF para no dar error
cols_financieras = [c for c in cols_financieras if c in df_clients.columns]
df_clients = df_clients.fillna(0, subset=cols_financieras)

# 3. GRUPO "CATEGÓRICO DESCONOCIDO" (Rellenar con 'Unknown')
# EDUCATION tiene 39k nulos. No podemos inventárnosla.
cols_categ_nulos = ["EDUCATION", "EMPLOYER_ORGANIZATION_TYPE", "MARITAL_STATUS"]
df_clients = df_clients.fillna("UNKNOWN", subset=cols_categ_nulos)

# 4. GRUPO "ANECDÓTICO" (Estrategia: Salvar al Cliente)
# Al ser poquísimos nulos, preferimos imputar para no perder la ficha del cliente.
# Para Numéricas (Installment): Usamos la MEDIANA (más robusta que la media)
# Calculamos la mediana aproximada (approxQuantile es muy eficiente en Spark)
cols_anecdoticas_num = ["INSTALLMENT", "FAMILY_SIZE"]
for col in cols_anecdoticas_num:
    # Calculamos la mediana de esa columna específica
    mediana = df_clients.stat.approxQuantile(col, [0.5], 0.01)[0]
    df_clients = df_clients.fillna(mediana, subset=[col])

print("Limpieza completada.")

Limpieza completada.


In [17]:
inspeccionar_datos(df_clients, "CLIENTS")


INSPECCIÓN DE: CLIENTS

Dimensiones: 162977 filas x 45 columnas

--- 1. Conteo de Nulos ---
-RECORD 0--------------------------
 CLIENT_ID                   | 0   
 NON_COMPLIANT_CONTRACT      | 0   
 NAME_PRODUCT_TYPE           | 0   
 GENDER                      | 0   
 TOTAL_INCOME                | 0   
 AMOUNT_PRODUCT              | 0   
 INSTALLMENT                 | 0   
 EDUCATION                   | 0   
 MARITAL_STATUS              | 0   
 HOME_SITUATION              | 0   
 REGION_SCORE                | 0   
 AGE_IN_YEARS                | 0   
 JOB_SENIORITY               | 0   
 HOME_SENIORITY              | 0   
 LAST_UPDATE                 | 0   
 OWN_INSURANCE_CAR           | 0   
 CAR_AGE                     | 0   
 FAMILY_SIZE                 | 0   
 REACTIVE_SCORING            | 0   
 PROACTIVE_SCORING           | 0   
 BEHAVIORAL_SCORING          | 0   
 DAYS_LAST_INFO_CHANGE       | 0   
 NUMBER_OF_PRODUCTS          | 0   
 OCCUPATION                  | 0   
 DIGITAL

In [18]:

# 5. INTEGRACIÓN (JOIN) - MODO NOMBRES ORIGINALES


print(" FASE 1: JOIN Y DIAGNÓSTICO (NOMBRES ORIGINALES)\n")

col_id = "CLIENT_ID" 

# --- A. SNAPSHOT (Último dato por cliente) ---
df_behav = df_behav.withColumn("DATE", F.to_date(F.col("DATE")))

# Ventana para quedarnos con el último registro
w = Window.partitionBy(col_id).orderBy(F.desc("DATE"))
df_behav_dedup = df_behav.withColumn("rank", F.row_number().over(w)) \
                         .filter(F.col("rank") == 1) \
                         .drop("rank")

# --- B. JOIN ---
print(f"Uniendo tablas por {col_id}...")
df_master = df_clients.join(df_behav_dedup, on=col_id, how="left")

# --- C. INTEGRIDAD ---
filas_clientes = df_clients.count()
filas_master = df_master.count()

if filas_master > filas_clientes:
    diff = filas_master - filas_clientes
    print(f"AVISO: Aún hay {diff} duplicados.")
else:
    print(f"CORRECTO: 1 Cliente = 1 Fila.")

# --- D. LIMPIEZA POST-JOIN (Nulos a 0) ---
# Tras el join, las columnas financieras de BEHAVIOURAL pueden tener nulos. 
# En el contexto bancario y transaccional, la ausencia de registro significa cantidad cero.
cols_financieras = [
    "CREDICT_CARD_BALANCE", "CREDIT_CARD_LIMIT", "NUMBER_INSTALMENTS", "NUMBER_DRAWINGS", 
    "NUMBER_DRAWINGS_ATM","CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", 
    "CREDIT_CARD_DRAWINGS_OTHER", "CREDIT_CARD_DRAWINGS", "CREDIT_CARD_PAYMENT"
]

cols_existentes = [c for c in cols_financieras if c in df_master.columns]
df_master = df_master.fillna(0, subset=cols_existentes)

# --- E. GESTIÓN DE FECHAS (ESTRATEGIA 'SENTINEL VALUE') ---
# En banca no se borran fechas, se imputa una fecha 'imposible' (1900-01-01).
if "DATE" in df_master.columns:
    print("Aplicando estándar bancario a fechas (Imputación 1900-01-01)...")
    
    # A) Primero calculamos la RECENCIA (Días desde último mov)
    #    Para esto usamos la fecha real antes de "taparla"
    max_date = df_master.agg(F.max("DATE")).collect()[0][0]
    
    df_master = df_master.withColumn(
        "KPI_DAYS_LAST_MOV", 
        F.datediff(F.lit(max_date), F.col("DATE"))
    )
    # Si es nulo (sin datos), ponemos 9999 días (inactivo histórico)
    df_master = df_master.fillna(9999, subset=["KPI_DAYS_LAST_MOV"])
    
    # B) Ahora aplicamos el SENTINEL VALUE a la columna original
    #    Usamos 'coalesce': Si DATE es null, pon 1900-01-01.
    df_master = df_master.withColumn(
        "DATE",
        F.coalesce(F.col("DATE"), F.lit("1900-01-01").cast("date"))
    )

# F. INFORMACIÓN DE LA TABLA 
print(" ESTADO ACTUAL (PRE-REDUCCIÓN)\n")
print(f"Dimensiones: {df_master.count()} filas x {len(df_master.columns)} columnas")
print("\nEsquema Actual ")
df_master.printSchema()

 FASE 1: JOIN Y DIAGNÓSTICO (NOMBRES ORIGINALES)

Uniendo tablas por CLIENT_ID...
CORRECTO: 1 Cliente = 1 Fila.
Aplicando estándar bancario a fechas (Imputación 1900-01-01)...
 ESTADO ACTUAL (PRE-REDUCCIÓN)

Dimensiones: 162977 filas x 59 columnas

Esquema Actual 
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = false)
 |-- EDUCATION: string (nullable = false)
 |-- MARITAL_STATUS: string (nullable = false)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = false)
 |-- HOME_SENIORITY: double (nullable = true)
 |-- LAST_UPDATE: double (nullable = true)
 |-- OWN_INSURANCE_CAR: string (nullable = true)
 |-- C

In [19]:

# 6. FEATURE ENGINEERING Y REDUCCIÓN FINAL

print("FASE 2: KPIs AVANZADOS Y LIMPIEZA FINAL\n ")

# --- A. CREACIÓN DE KPIs (Información de Negocio) ---
print("Generando KPIs estratégicos...")

# 1. KPI Gasto Total Tarjeta (Suma de los parciales)
df_master = df_master.withColumn(
    "KPI_TOTAL_SPEND",
    F.col("CREDIT_CARD_DRAWINGS_ATM") + F.col("CREDIT_CARD_DRAWINGS_POS") + F.col("CREDIT_CARD_DRAWINGS_OTHER")
)

# 2. KPI Ratio Endeudamiento (Deuda / Ingresos)
# Sumamos 1 al ingreso para evitar divisiones por cero
df_master = df_master.withColumn(
    "KPI_DEBT_RATIO",
    F.round(F.col("CREDICT_CARD_BALANCE") / (F.col("TOTAL_INCOME") + 1), 4)
)

# 3. KPI Grupo de Edad (Simplificación demográfica)
df_master = df_master.withColumn(
    "KPI_AGE_GROUP",
    F.when(F.col("AGE_IN_YEARS") < 30, "Joven")
     .when(F.col("AGE_IN_YEARS") < 50, "Adulto")
     .otherwise("Senior")
)

# 4. KPI Volatilidad de Préstamos (Max - Min)
df_master = df_master.withColumn(
    "KPI_LOAN_VOLATILITY",
    F.col("LOAN_CREDIT_GRANTED_MAX") - F.col("LOAN_CREDIT_GRANTED_MIN")
)

# 5. KPI Ratio de Aprobación (Lo que le dieron / Lo que pidió)
# Indica la confianza del banco en el cliente
df_master = df_master.withColumn(
    "KPI_APPROVAL_RATIO",
    F.round(F.col("LOAN_CREDIT_GRANTED_SUM") / (F.col("LOAN_APPLICATION_AMOUNT_SUM") + 1), 2)
)

# 6. KPI Tasa de Rechazo (Solicitudes denegadas / Total)
# Resume los estados conflictivos
df_master = df_master.withColumn(
    "KPI_DENIAL_RATE",
    F.round(F.col("NUM_STATUS_DENIED") / 
            (F.col("NUM_STATUS_AUTHORIZED") + F.col("NUM_STATUS_DENIED") + F.col("NUM_STATUS_ANNULLED") + 1), 2)
)

# --- B. REDUCCIÓN DE COLUMNAS (Limpieza de "Grasa") ---
print("Eliminando columnas redundantes e inútiles...")

cols_a_borrar = [
    # 1. Redundantes de Tarjeta (Sustituidas por KPI_TOTAL_SPEND)
    "CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", "CREDIT_CARD_DRAWINGS_OTHER", 
    "CREDIT_CARD_DRAWINGS", # Dato duplicado original
    
    # 2. Redundantes de Préstamos (Sustituidas por KPI_LOAN_VOLATILITY y APPROVAL)
    "LOAN_ANNUITY_PAYMENT_MAX", "LOAN_ANNUITY_PAYMENT_MIN",
    "LOAN_APPLICATION_AMOUNT_MAX", "LOAN_APPLICATION_AMOUNT_MIN",
    "LOAN_CREDIT_GRANTED_MAX", "LOAN_CREDIT_GRANTED_MIN",
    "LOAN_VARIABLE_RATE_MAX", "LOAN_VARIABLE_RATE_MIN",

    # 3. Basura Técnica y Duplicados
    "CONTRACT_ID",  # ID interno técnico (inútil para negocio)
    "CURRENCY"      # Columna duplicada por el Join (borramos ambas)
    ]

# Borrado seguro (solo las que existan)
cols_finales_borrar = [c for c in cols_a_borrar if c in df_master.columns]
df_master = df_master.drop(*cols_finales_borrar)

# --- C. DIMENSIONES FINALES ---

print("ESTADO FINAL OPTIMIZADO\n")
print(f"Dimensiones: {df_master.count()} filas x {len(df_master.columns)} columnas")
print(f"(Se han eliminado {len(cols_finales_borrar)} columnas redundantes y creado 6 KPIs estratégicos)")

# --- D. GUARDADO FINAL ---
ruta_final = OUTPUT_PATH + "Master_FinPlus.parquet"
df_master.write.mode("overwrite").parquet(ruta_final)
print(f"\nGuardado archivo en: {ruta_final}.")

# --- E. GUARDADO EN PARQUET ÚNICO ---

df_pandas = df_master.toPandas()

nombre_archivo = "Master_FinPlus_Final.parquet"
ruta_completa = OUTPUT_PATH + nombre_archivo

# Necesitas tener instalada la librería pyarrow o fastparquet (suele venir en Docker)
df_pandas.to_parquet(ruta_completa, index=False)

print(f"\nGuardado.")
print(f" Ruta: {ruta_completa}")

df_master.printSchema()
inspeccionar_datos(df_master, "MASTER BOARD")


FASE 2: KPIs AVANZADOS Y LIMPIEZA FINAL
 
Generando KPIs estratégicos...
Eliminando columnas redundantes e inútiles...
ESTADO FINAL OPTIMIZADO

Dimensiones: 162977 filas x 50 columnas
(Se han eliminado 14 columnas redundantes y creado 6 KPIs estratégicos)

Guardado archivo en: /home/jovyan/work/data/curated/Master_FinPlus.parquet.

Guardado.
 Ruta: /home/jovyan/work/data/curated/Master_FinPlus_Final.parquet
root
 |-- CLIENT_ID: string (nullable = true)
 |-- NON_COMPLIANT_CONTRACT: integer (nullable = true)
 |-- NAME_PRODUCT_TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- TOTAL_INCOME: double (nullable = true)
 |-- AMOUNT_PRODUCT: double (nullable = true)
 |-- INSTALLMENT: double (nullable = false)
 |-- EDUCATION: string (nullable = false)
 |-- MARITAL_STATUS: string (nullable = false)
 |-- HOME_SITUATION: string (nullable = true)
 |-- REGION_SCORE: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- JOB_SENIORITY: double (nullable = false

In [20]:
def auditar_outliers(df, cols_numericas):
    
    print(f" VERIFICACIÓN DE OUTLIERS (Método IQR)")
    
    
    for col in cols_numericas:
        # 1. Calculamos Cuartiles (25% y 75%)
        quantiles = df.stat.approxQuantile(col, [0.25, 0.75], 0.01)
        q1, q3 = quantiles[0], quantiles[1]
        iqr = q3 - q1
        
        # 2. Definimos límites (Bigotes del Boxplot)
        limite_inf = q1 - 1.5 * iqr
        limite_sup = q3 + 1.5 * iqr
        
        # 3. Contamos cuántos se salen
        outliers = df.filter((F.col(col) < limite_inf) | (F.col(col) > limite_sup))
        num_outliers = outliers.count()
        
        if num_outliers > 0:
            print(f"\n Columna: {col}")
            print(f"   Rango Normal: [{limite_inf:.2f}  a  {limite_sup:.2f}]")
            print(f"   Outliers detectados: {num_outliers} filas")
            
            # Mostramos los valores más extremos para ver si son errores o VIPs
            print(f"   Ejemplos (Top Extremos):")
            outliers.select(col).orderBy(F.desc(col)).show(3)
        else:
            print(f"\n Columna: {col} -> Sin outliers estadísticos.")

# Definimos las columnas numéricas críticas para analizar
cols_analisis = ["TOTAL_INCOME", "AGE_IN_YEARS", "AMOUNT_PRODUCT", "CREDICT_CARD_BALANCE"]

# Ejecutamos
auditar_outliers(df_master, cols_analisis)

 VERIFICACIÓN DE OUTLIERS (Método IQR)

 Columna: TOTAL_INCOME
   Rango Normal: [-270.00  a  4050.00]
   Outliers detectados: 7409 filas
   Ejemplos (Top Extremos):
+------------+
|TOTAL_INCOME|
+------------+
|   1404000.0|
|   216001.08|
|    108000.0|
+------------+
only showing top 3 rows


 Columna: AGE_IN_YEARS -> Sin outliers estadísticos.

 Columna: AMOUNT_PRODUCT
   Rango Normal: [-6455.70  a  19399.50]
   Outliers detectados: 3465 filas
   Ejemplos (Top Extremos):
+--------------+
|AMOUNT_PRODUCT|
+--------------+
|     48486.195|
|     48486.195|
|     48486.195|
+--------------+
only showing top 3 rows


 Columna: CREDICT_CARD_BALANCE
   Rango Normal: [0.00  a  0.00]
   Outliers detectados: 16336 filas
   Ejemplos (Top Extremos):
+--------------------+
|CREDICT_CARD_BALANCE|
+--------------------+
|            14526.13|
|            12122.07|
|            11500.21|
+--------------------+
only showing top 3 rows



### Conversión de variables categóricas a numéricas con Label Encoding

In [21]:
# ==========================================
# 8. TRANSFORMACIÓN FINAL: LABEL ENCODING PARA ML
# ==========================================
print("\n" + "="*40)
print("FASE 4: PREPARACIÓN PARA MODELOS (STRING INDEXER)")
print("="*40)

# 1. Detectar columnas de texto automáticamente
# Definimos la "Lista Negra" de columnas que NO son características (IDs, Fechas, etc.)
cols_excluir = ["CLIENT_ID", "CONTRACT_ID", "DATE", "date"] 

# Buscamos todas las columnas de tipo String que no estén en la lista de exclusión
categ_cols = [f.name for f in df_master.schema.fields 
              if isinstance(f.dataType, T.StringType) and f.name not in cols_excluir]

print(f"Columnas categóricas detectadas para transformar ({len(categ_cols)}):")
print(categ_cols)

# 2. Configurar el StringIndexer (Label Encoding)
# - handleInvalid="keep": Si en el futuro aparece una categoría nueva no vista, crea un índice extra en vez de dar error.
# - outputCol: Crea nuevas columnas terminadas en "_IDX".
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_IDX", handleInvalid="keep") for col in categ_cols]

# 3. Ejecutar la transformación (Pipeline)
# Usamos un Pipeline para aplicar todos los indexers de golpe de forma eficiente
pipeline = Pipeline(stages=indexers)
model = pipeline.fit(df_master)
df_model_ready = model.transform(df_master)

# 4. Selección Final: Solo columnas numéricas
# Nos quedamos con las numéricas originales + los nuevos índices (_IDX).
# Descartamos las columnas de texto originales y los IDs.

# Lista de columnas numéricas originales (Double, Integer, Long)
numeric_cols_orig = [f.name for f in df_master.schema.fields 
                     if isinstance(f.dataType, (T.DoubleType, T.IntegerType, T.LongType))]

# Lista de las nuevas columnas indexadas
idx_cols = [f"{col}_IDX" for col in categ_cols]

# Ensamblamos la lista final
cols_definitivas = numeric_cols_orig + idx_cols
df_final_numeric = df_model_ready.select(cols_definitivas)

# 5. RESULTADO Y GUARDADO
print("DATASET FINAL LISTO PARA ENTRENAMIENTO")

# Mostrar un ejemplo de cómo quedan los datos
print("\nEjemplo de datos transformados:")
df_final_numeric.show(5)

# --- E. GUARDADO EN PARQUET ÚNICO ---
# 1. Convertimos de Spark a Pandas
print("Convirtiendo a formato local (Pandas)...")
df_pandas2 = df_final_numeric.toPandas()

# 2. Definimos nombre y ruta
nombre_archivo = "Master_Model_FinPlus.parquet"
ruta_completa = OUTPUT_PATH + nombre_archivo

# 3. Guardamos el archivo físico
# index=False evita que se guarde el número de fila como una columna extra
df_pandas2.to_parquet(ruta_completa, index=False)

print(f"\n Archivo único guardado en: {ruta_completa}.")

df_master.printSchema()
inspeccionar_datos(df_final_numeric, "MASTER MODEL")



FASE 4: PREPARACIÓN PARA MODELOS (STRING INDEXER)
Columnas categóricas detectadas para transformar (10):
['NAME_PRODUCT_TYPE', 'GENDER', 'EDUCATION', 'MARITAL_STATUS', 'HOME_SITUATION', 'OWN_INSURANCE_CAR', 'OCCUPATION', 'HOME_OWNER', 'EMPLOYER_ORGANIZATION_TYPE', 'KPI_AGE_GROUP']
DATASET FINAL LISTO PARA ENTRENAMIENTO

Ejemplo de datos transformados:
+----------------------+------------+--------------+-----------+------------------+------------------+-------------+--------------+-----------+-------+-----------+------------------+------------------+------------------+---------------------+------------------+--------------+---------------------+------------------------+---------------------------+-----------------------+-------------------+---------------------+-----------------+-------------------+----------------+--------------------+-----------------+-------------------+-------------------+---------------+------------------+-----------------+---------------+--------------+----------

# INDICADORES Y ANÁLISIS DE COMPORTAMIENTO

### ACTIVIDAD CLIENTE

- Cálculo de métricas de actividad:
    - Recencia (R)

    - Frecuencia (F)

    - Intensidad (I)

- Ventanas de actividad (30/90/180 días)

- Meses activos

- Clasificación de actividad (Alta / Media / Baja)

- Interpretación clara de negocio


In [22]:
# ==========================================
# 1. CARGAR DATOS YA PROCESADOS
# ==========================================
DATA_PATH = "/home/jovyan/work/data/"

beh = spark.read.parquet(DATA_PATH + "BEHAVIOURAL.parquet")
df_master = spark.read.parquet(DATA_PATH + "curated/Master_FinPlus.parquet")

beh = beh.withColumn("DATE", F.to_date("DATE", "yyyy-MM-dd"))
max_date = beh.agg(F.max("DATE")).first()[0]




In [23]:
# ==========================================
# 2. MÉTRICAS DE ACTIVIDAD
# ==========================================

# --- RECENCIA ---
recencia_df = beh.groupBy("CLIENT_ID").agg(
    F.max("DATE").alias("last_activity_date")
).withColumn(
    "RECENCY_DAYS", F.datediff(F.lit(max_date), F.col("last_activity_date"))
)

print("\n================ RECENCIA (R) ================\n")
print("Interpretación: número de días desde la última actividad del cliente.")
recencia_df.describe("RECENCY_DAYS").show()

print("\n--- Top 10 clientes más recientes ---")
recencia_df.orderBy(F.col("RECENCY_DAYS").asc()).show(10, truncate=False)

print("\n--- Top 10 clientes más abandonados ---")
recencia_df.orderBy(F.col("RECENCY_DAYS").desc()).show(10, truncate=False)




================ RECENCIA (R) ================

Interpretación: número de días desde la última actividad del cliente.
+-------+------------------+
|summary|      RECENCY_DAYS|
+-------+------------------+
|  count|             46046|
|   mean|15.444794336098685|
| stddev| 21.54619528266414|
|    min|                 0|
|    max|                93|
+-------+------------------+


--- Top 10 clientes más recientes ---
+------------+------------------+------------+
|CLIENT_ID   |last_activity_date|RECENCY_DAYS|
+------------+------------------+------------+
|ES182303796D|2021-12-31        |0           |
|ES182245752Y|2021-12-31        |0           |
|ES182293250V|2021-12-31        |0           |
|ES182245476M|2021-12-31        |0           |
|ES182112694Y|2021-12-31        |0           |
|ES182189508S|2021-12-31        |0           |
|ES182433571C|2021-12-31        |0           |
|ES182232062V|2021-12-31        |0           |
|ES182378189N|2021-12-31        |0           |
|ES182279031S|20

In [24]:

# --- FRECUENCIA ---
frecuencia_df = beh.groupBy("CLIENT_ID").agg(
    F.count("*").alias("FREQUENCY_COUNT")
)

print("\n================ FRECUENCIA (F) ================\n")
print("Interpretación: cuántos movimientos totales ha realizado el cliente.")
frecuencia_df.describe("FREQUENCY_COUNT").show()

print("\n--- Top 10 clientes con mayor frecuencia ---")
frecuencia_df.orderBy(F.col("FREQUENCY_COUNT").desc()).show(10, truncate=False)


================ FRECUENCIA (F) ================

Interpretación: cuántos movimientos totales ha realizado el cliente.
+-------+-----------------+
|summary|  FREQUENCY_COUNT|
+-------+-----------------+
|  count|            46046|
|   mean|37.45936672023628|
| stddev|33.78619079695795|
|    min|                1|
|    max|              192|
+-------+-----------------+


--- Top 10 clientes con mayor frecuencia ---
+------------+---------------+
|CLIENT_ID   |FREQUENCY_COUNT|
+------------+---------------+
|ES182186401T|192            |
|ES182128827G|129            |
|ES182192917N|126            |
|ES182283225D|122            |
|ES182378495D|122            |
|ES182155668A|121            |
|ES182253915Y|121            |
|ES182146380G|121            |
|ES182267366X|120            |
|ES182210848C|120            |
+------------+---------------+
only showing top 10 rows



In [25]:
# --- INTENSIDAD ---
beh = beh.withColumn(
    "KPI_TOTAL_SPEND",
    F.coalesce(F.col("CREDIT_CARD_DRAWINGS_ATM"), F.lit(0)) +
    F.coalesce(F.col("CREDIT_CARD_DRAWINGS_POS"), F.lit(0)) +
    F.coalesce(F.col("CREDIT_CARD_DRAWINGS_OTHER"), F.lit(0))
)

intensidad_df = beh.groupBy("CLIENT_ID").agg(
    F.avg("KPI_TOTAL_SPEND").alias("INTENSITY_AVG_SPEND")
)

print("\n================ INTENSIDAD (I) ================\n")
print("Interpretación: gasto promedio por movimiento del cliente.")
intensidad_df.describe("INTENSITY_AVG_SPEND").show()

print("\n--- Top 10 clientes con mayor intensidad ---")
intensidad_df.orderBy(F.col("INTENSITY_AVG_SPEND").desc()).show(10, truncate=False)




================ INTENSIDAD (I) ================

Interpretación: gasto promedio por movimiento del cliente.
+-------+-------------------+
|summary|INTENSITY_AVG_SPEND|
+-------+-------------------+
|  count|              46046|
|   mean| 165.34456214658363|
| stddev|  310.5056279798483|
|    min|                0.0|
|    max|  9629.483333333334|
+-------+-------------------+


--- Top 10 clientes con mayor intensidad ---
+------------+-------------------+
|CLIENT_ID   |INTENSITY_AVG_SPEND|
+------------+-------------------+
|ES182227107Z|9629.483333333334  |
|ES182354235A|8642.13            |
|ES182436756T|7395.070000000001  |
|ES182156500S|5901.773846153846  |
|ES182366160A|5899.592142857144  |
|ES182390308Q|5731.825           |
|ES182247132U|5427.0             |
|ES182396618L|5346.0             |
|ES182129030R|5245.332173913043  |
|ES182250483A|5186.4158333333335 |
+------------+-------------------+
only showing top 10 rows



In [26]:

# ==========================================
# 3. ACTIVIDAD 30 / 90 / 180 DÍAS
# ==========================================
beh_windows = beh.withColumn(
    "DAYS_FROM_REF", F.datediff(F.lit(max_date), F.col("DATE"))
)

ventanas_df = beh_windows.groupBy("CLIENT_ID").agg(
    F.sum(F.when(F.col("DAYS_FROM_REF") <= 30, 1).otherwise(0)).alias("ACTIVITY_30D"),
    F.sum(F.when(F.col("DAYS_FROM_REF") <= 90, 1).otherwise(0)).alias("ACTIVITY_90D"),
    F.sum(F.when(F.col("DAYS_FROM_REF") <= 180, 1).otherwise(0)).alias("ACTIVITY_180D")
)

print("\n================ ACTIVIDAD EN VENTANAS ================\n")
print("Interpretación: cuántas interacciones ha tenido el cliente en los últimos X días.\n")

print("--- Top 10 actividad últimos 30 días ---")
ventanas_df.orderBy(F.col("ACTIVITY_30D").desc()).show(10, truncate=False)

print("--- Top 10 actividad últimos 90 días ---")
ventanas_df.orderBy(F.col("ACTIVITY_90D").desc()).show(10, truncate=False)

print("--- Top 10 actividad últimos 180 días ---")
ventanas_df.orderBy(F.col("ACTIVITY_180D").desc()).show(10, truncate=False)




================ ACTIVIDAD EN VENTANAS ================

Interpretación: cuántas interacciones ha tenido el cliente en los últimos X días.

--- Top 10 actividad últimos 30 días ---
+------------+------------+------------+-------------+
|CLIENT_ID   |ACTIVITY_30D|ACTIVITY_90D|ACTIVITY_180D|
+------------+------------+------------+-------------+
|ES182403907W|2           |6           |12           |
|ES182334710B|2           |6           |12           |
|ES182210848C|2           |6           |12           |
|ES182347896Q|2           |6           |12           |
|ES182243027C|2           |6           |11           |
|ES182407401Q|2           |6           |12           |
|ES182127891W|2           |5           |8            |
|ES182150696Z|2           |6           |12           |
|ES182100594F|2           |4           |7            |
|ES182283225D|2           |6           |12           |
+------------+------------+------------+-------------+
only showing top 10 rows

--- Top 10 actividad ú

In [27]:

# ==========================================
# 4. MESES ACTIVOS
# ==========================================
beh_month = beh.withColumn(
    "YEAR_MONTH", F.date_format("DATE", "yyyy-MM")
)

meses_activos_df = beh_month.groupBy("CLIENT_ID").agg(
    F.countDistinct("YEAR_MONTH").alias("ACTIVE_MONTHS")
)

print("\n================ MESES ACTIVOS ================\n")
meses_activos_df.orderBy(F.col("ACTIVE_MONTHS").desc()).show(10, truncate=False)




================ MESES ACTIVOS ================

+------------+-------------+
|CLIENT_ID   |ACTIVE_MONTHS|
+------------+-------------+
|ES182123143P|96           |
|ES182173222Z|96           |
|ES182230930K|96           |
|ES182167286W|96           |
|ES182243311L|96           |
|ES182233737B|96           |
|ES182168740A|96           |
|ES182258895P|96           |
|ES182319130A|96           |
|ES182350305O|96           |
+------------+-------------+
only showing top 10 rows



In [28]:

# ==========================================
# 5. COMBINAR TODAS LAS MÉTRICAS
# ==========================================
activity_metrics = (
    recencia_df
    .join(frecuencia_df, "CLIENT_ID", "left")
    .join(intensidad_df, "CLIENT_ID", "left")
    .join(ventanas_df, "CLIENT_ID", "left")
    .join(meses_activos_df, "CLIENT_ID", "left")
)


In [29]:

# ==========================================
# 6. SEGMENTACIÓN DE ACTIVIDAD
# ==========================================
activity_metrics = activity_metrics.withColumn(
    "ACTIVITY_SEGMENT",
    F.when((F.col("RECENCY_DAYS") <= 30) & (F.col("FREQUENCY_COUNT") > 5), "Alta")
     .when((F.col("RECENCY_DAYS") <= 90) & (F.col("FREQUENCY_COUNT") > 2), "Media")
     .otherwise("Baja")
)

print("\n================ SEGMENTACIÓN FINAL ================\n")
activity_metrics.groupBy("ACTIVITY_SEGMENT").count().show()

print("\n--- Muestra de clientes segmentados ---")
activity_metrics.select(
    "CLIENT_ID", "RECENCY_DAYS", "FREQUENCY_COUNT", 
    "INTENSITY_AVG_SPEND", "ACTIVITY_SEGMENT"
).orderBy("ACTIVITY_SEGMENT").show(20, truncate=False)




================ SEGMENTACIÓN FINAL ================

+----------------+-----+
|ACTIVITY_SEGMENT|count|
+----------------+-----+
|            Alta|26002|
|           Media|18448|
|            Baja| 1596|
+----------------+-----+


--- Muestra de clientes segmentados ---
+------------+------------+---------------+-------------------+----------------+
|CLIENT_ID   |RECENCY_DAYS|FREQUENCY_COUNT|INTENSITY_AVG_SPEND|ACTIVITY_SEGMENT|
+------------+------------+---------------+-------------------+----------------+
|ES182222478Q|0           |19             |588.2236842105264  |Alta            |
|ES182325278Y|0           |9              |625.4711111111111  |Alta            |
|ES182133642C|0           |85             |31.129411764705882 |Alta            |
|ES182245476M|0           |10             |0.0                |Alta            |
|ES182413873B|0           |91             |106.26186813186811 |Alta            |
|ES182245752Y|0           |26             |0.0                |Alta            |

In [30]:

# ==========================================
# 7. UNIR A MASTER FINAL
# ==========================================
df_final = df_master.join(activity_metrics, "CLIENT_ID", "left")
df_final.write.mode("overwrite").parquet(DATA_PATH + "Master_FinPlus_Activity.parquet")

print("\nArchivo guardado correctamente en: Master_FinPlus_Activity.parquet")



Archivo guardado correctamente en: Master_FinPlus_Activity.parquet


### VALOR ECONÓMICO

- ECONOMIC VALUE SCORE (E-SCORE)
    - Total gastado

    - Volumen de compras

    - Número de compras

    - Rentabilidad generada

- MÉTRICAS POR CLIENTE
    - Gasto total

    - Ticket medio

    - Varianza del gasto

    - Compra máxima

    - Compra mínima

    - Ratio de recurrecia económica

    - Índices de "customer value" (cuartiles o percentiles)

- SEGMENTACIÓN
    - Alto valor

    - Medio

    - Bajo

In [31]:
# ------------------------------------------------------------
# 1. MÉTRICAS ECONÓMICAS BÁSICAS POR CLIENTE
# ------------------------------------------------------------

econ = beh.groupBy("CLIENT_ID").agg(
    # total gastado, volumen de compras
    F.sum("KPI_TOTAL_SPEND").alias("TOTAL_SPEND"), 
    
    # rentabilidad generada (usamos gasto promedio como proxy de valor por transacción)
    F.avg("KPI_TOTAL_SPEND").alias("AVG_TICKET"), # Ticket medio
    
    # varianza del gasto (usamos stddev como medida de dispersión/volatilidad)
    F.stddev("KPI_TOTAL_SPEND").alias("SPEND_STDDEV"),
    
    # compra máxima
    F.max("KPI_TOTAL_SPEND").alias("MAX_PURCHASE"),
    
    # compra mínima
    F.min("KPI_TOTAL_SPEND").alias("MIN_PURCHASE"),
    
    # numero de compras
    F.count("*").alias("NUM_TRANSACTIONS")
)

print("\n================ 1. MÉTRICAS ECONÓMICAS BÁSICAS ================\n")
print("Métricas: Gasto Total, Ticket Medio, Varianza, Max/Min Compra, Nº Transacciones.")

print("✔ Resumen descriptivo del gasto total (TOTAL_SPEND)")
econ.select("TOTAL_SPEND").describe().show()

print("\n✔ Top 10 clientes con mayor gasto total:")
econ.orderBy(F.col("TOTAL_SPEND").desc()).show(10, truncate=False)

print("\n✔ Ticket medio (Top 10 más altos):")
econ.orderBy(F.col("AVG_TICKET").desc()).show(10, truncate=False)

print("\n✔ Distribución de número de transacciones:")
econ.select("NUM_TRANSACTIONS").describe().show()


================ 1. MÉTRICAS ECONÓMICAS BÁSICAS ================

Métricas: Gasto Total, Ticket Medio, Varianza, Max/Min Compra, Nº Transacciones.
✔ Resumen descriptivo del gasto total (TOTAL_SPEND)
+-------+-----------------+
|summary|      TOTAL_SPEND|
+-------+-----------------+
|  count|            46046|
|   mean|3348.860903227201|
| stddev|5342.655025746742|
|    min|              0.0|
|    max|        192187.19|
+-------+-----------------+


✔ Top 10 clientes con mayor gasto total:
+------------+-----------------+------------------+------------------+------------+------------+----------------+
|CLIENT_ID   |TOTAL_SPEND      |AVG_TICKET        |SPEND_STDDEV      |MAX_PURCHASE|MIN_PURCHASE|NUM_TRANSACTIONS|
+------------+-----------------+------------------+------------------+------------+------------+----------------+
|ES182348429U|192187.19        |5057.557631578948 |1830.513040213463 |8507.65     |186.6       |38              |
|ES182227107Z|144442.25        |9629.483333333334

In [32]:
# ------------------------------------------------------------
# 2. RATIO DE RECURRENCIA ECONÓMICA
# ------------------------------------------------------------

econ = econ.withColumn(
    "SPEND_RECURRENCE_RATIO",
    # Ratio: Gasto Total / (Nº Transacciones + 1). Cuanto más alto, mayor el gasto promedio por evento.
    F.round(F.col("TOTAL_SPEND") / (F.col("NUM_TRANSACTIONS") + 1), 2)
)

print("\n================ 2. RATIO DE RECURRENCIA ECONÓMICA ================\n")
print("Ratio: Gasto Total / (Nº Transacciones + 1).")
econ.select("CLIENT_ID", "TOTAL_SPEND", "NUM_TRANSACTIONS", "SPEND_RECURRENCE_RATIO") \
    .orderBy(F.col("SPEND_RECURRENCE_RATIO").desc()) \
    .show(10, truncate=False)




================ 2. RATIO DE RECURRENCIA ECONÓMICA ================

Ratio: Gasto Total / (Nº Transacciones + 1).
+------------+------------------+----------------+----------------------+
|CLIENT_ID   |TOTAL_SPEND       |NUM_TRANSACTIONS|SPEND_RECURRENCE_RATIO|
+------------+------------------+----------------+----------------------+
|ES182227107Z|144442.25         |15              |9027.64               |
|ES182436756T|51765.490000000005|7               |6470.69               |
|ES182366160A|82594.29000000001 |14              |5506.29               |
|ES182156500S|76723.06          |13              |5480.22               |
|ES182129030R|120642.64         |23              |5026.78               |
|ES182348429U|192187.19         |38              |4927.88               |
|ES182446412N|72112.26          |14              |4807.48               |
|ES182250483A|62236.99          |12              |4787.46               |
|ES182358570I|79456.63          |16              |4673.92              

In [33]:
# ------------------------------------------------------------
# 3. CLASIFICACIÓN DE VALOR ECONÓMICO (E-SCORE: Alto, Medio, Bajo)
# ------------------------------------------------------------

# Índices de "customer value" (cuartiles o percentiles)
# Usaremos los tertiles (33% y 66%) de TOTAL_SPEND para la segmentación.
quantiles = econ.approxQuantile("TOTAL_SPEND", [0.33, 0.66], 0.01)
p33, p66 = quantiles

econ = econ.withColumn(
    "ECONOMIC_VALUE_CLASS",
    F.when(F.col("TOTAL_SPEND") >= p66, "ALTO VALOR")
     .when(F.col("TOTAL_SPEND") >= p33, "VALOR MEDIO")
     .otherwise("BAJO VALOR")
)

print("\n================ 3. SEGMENTACIÓN DE VALOR ECONÓMICO (E-SCORE) ================\n")
print("Segmentación (Alto, Medio, Bajo) basada en los tertiles de Gasto Total (TOTAL_SPEND).")

print("\n✔ Segmentación económica completada:")
econ.groupBy("ECONOMIC_VALUE_CLASS").count().show()

print("\n✔ Ejemplo de clientes por cada categoría:")
econ.select("CLIENT_ID", "TOTAL_SPEND", "ECONOMIC_VALUE_CLASS") \
    .orderBy(F.col("TOTAL_SPEND").desc()).show(20, truncate=False)




================ 3. SEGMENTACIÓN DE VALOR ECONÓMICO (E-SCORE) ================

Segmentación (Alto, Medio, Bajo) basada en los tertiles de Gasto Total (TOTAL_SPEND).

✔ Segmentación económica completada:
+--------------------+-----+
|ECONOMIC_VALUE_CLASS|count|
+--------------------+-----+
|          ALTO VALOR|15876|
|         VALOR MEDIO|15245|
|          BAJO VALOR|14925|
+--------------------+-----+


✔ Ejemplo de clientes por cada categoría:
+------------+-----------------+--------------------+
|CLIENT_ID   |TOTAL_SPEND      |ECONOMIC_VALUE_CLASS|
+------------+-----------------+--------------------+
|ES182348429U|192187.19        |ALTO VALOR          |
|ES182227107Z|144442.25        |ALTO VALOR          |
|ES182129030R|120642.64        |ALTO VALOR          |
|ES182117163D|100618.2         |ALTO VALOR          |
|ES182403153O|99603.0          |ALTO VALOR          |
|ES182262674U|91530.0          |ALTO VALOR          |
|ES182446227G|87831.59000000003|ALTO VALOR          |
|ES18211

In [34]:
# ------------------------------------------------------------
# 4. UNIÓN AL MASTER
# ------------------------------------------------------------

df_master = df_master.join(econ, on="CLIENT_ID", how="left")

print("\n================ 4. UNIÓN AL MASTER ================\n")
print("Métricas económicas añadidas al master correctamente.")
print(f"Nuevo total de columnas: {len(df_master.columns)}\n")


================ 4. UNIÓN AL MASTER ================

Métricas económicas añadidas al master correctamente.
Nuevo total de columnas: 58



### INTERACCIÓN Y FIDELIDAD

In [41]:
## Análisis y Cálculo de Métricas de Interacción y Fidelidad (I-Score)


# ==========================================
# 0. PREPARACIÓN DE DATOS BASE
# ASUMIMOS:
# - 'beh' (DataFrame BEHAVIOURAL) está cargado y limpio.
# - 'df_master' (DataFrame CLIENTS/Master) está cargado.
# ==========================================

# Rellenar Nulos en las columnas de gasto y pago.
# Es fundamental tratar los valores nulos como 0 antes de realizar sumas o calcular ratios para evitar errores (NaN) e inconsistencias.
beh = beh.fillna(0, subset=["CREDIT_CARD_DRAWINGS_ATM", "CREDIT_CARD_DRAWINGS_POS", "CREDIT_CARD_DRAWINGS_OTHER","CREDIT_CARD_PAYMENT", "CREDIT_CARD_DRAWINGS", "NUMBER_DRAWINGS"])

1. Métricas de Uso de Canales y Fidelidad Transaccional

1.1. Cálculo de Gasto Total y Agregación

Se calcula el gasto total por transacción (KPI_TOTAL_SPEND) y luego se agregan todas las transacciones por cliente para obtener los totales (sumas) de gasto por canal y de pagos.

In [43]:
# --- 1.1. KPI de Gasto Total (Total Drawings) por Transacción ---
# Se crea la columna de Gasto Total sumando los gastos por cada canal.
if "KPI_TOTAL_SPEND" not in beh.columns:
    beh = beh.withColumn("KPI_TOTAL_SPEND", F.col("CREDIT_CARD_DRAWINGS_ATM") + F.col("CREDIT_CARD_DRAWINGS_POS") + F.col("CREDIT_CARD_DRAWINGS_OTHER"))


# --- 1.2. Métrica de Repuesta/Fidelidad por CLIENTE (Agregación) ---

interaccion_df = beh.groupBy("CLIENT_ID").agg(
    # Suma de Gasto por Canales
    F.sum("CREDIT_CARD_DRAWINGS_ATM").alias("SPEND_ATM_SUM"),
    F.sum("CREDIT_CARD_DRAWINGS_POS").alias("SPEND_POS_SUM"),
    F.sum("CREDIT_CARD_DRAWINGS_OTHER").alias("SPEND_OTHER_SUM"),

    # Suma de Gasto Total y Pago Total
    F.sum("KPI_TOTAL_SPEND").alias("TOTAL_DRAWINGS_SPEND"),
    F.sum("CREDIT_CARD_PAYMENT").alias("TOTAL_PAYMENTS") # Dinero que paga (amortiza)
)

1.2. Cálculo de Ratios (Manejo de División por Cero)
Para evitar errores de división por cero (que ocurrían si TOTAL_DRAWINGS_SPEND era 0), se utiliza la técnica de sumar + 1 al denominador en todos los cálculos de ratio.

In [44]:
# Cálculo de Ratios
# NOTA: Se mantiene la técnica de sumar +1 al denominador para evitar división por cero (y NaN/Inf)
interaccion_df = interaccion_df.withColumn(
    # Ratio ATM: Proporción del gasto en ATM sobre el Gasto Total.
    "CHANNEL_ATM_RATIO", F.round(F.col("SPEND_ATM_SUM") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    # Ratio POS: Proporción del gasto en POS sobre el Gasto Total.
    "CHANNEL_POS_RATIO", F.round(F.col("SPEND_POS_SUM") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    # Ratio OTHER: Proporción del gasto en Otros sobre el Gasto Total.
    "CHANNEL_OTHER_RATIO", F.round(F.col("SPEND_OTHER_SUM") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
).withColumn(
    # Métrica de Fidelidad (Gasto vs. Pago): Mide el uso responsable. Valores altos implican que el cliente paga mucho en relación a su gasto.
    "PAYMENT_FIDELITY_RATIO", F.round(F.col("TOTAL_PAYMENTS") / (F.col("TOTAL_DRAWINGS_SPEND") + 1), 4)
)

1.3. Top 10 Clientes Fieles

Este paso implementa la corrección para obtener un Top 10 de clientes fieles que realmente están activos y gastando, lo que garantiza que los ratios de canal sean informativos y no 0.0.

In [ ]:
# --- 1.3. Top 10 Clientes Fieles (y Activos) ---
print("\n✔ Top 10 Clientes con Mayor Fidelidad de Pago (FILTRANDO los clientes con GASTO CERO):")
# FILTRO CRÍTICO: Excluimos a los clientes cuyo gasto total es 0. 
# Esto elimina los 'outliers' de fidelidad artificial y asegura que los ratios de canal (ATM, POS, Other) sean significativos (> 0).
interaccion_df.filter(F.col("TOTAL_DRAWINGS_SPEND") > 0)\
    .orderBy(F.col("PAYMENT_FIDELITY_RATIO").desc())\
    .show(10, truncate=False)



✔ Top 10 Clientes con Mayor Fidelidad de Pago (FILTRANDO los clientes con GASTO CERO):
+------------+-------------+-------------+---------------+--------------------+------------------+-----------------+-----------------+-------------------+----------------------+
|CLIENT_ID   |SPEND_ATM_SUM|SPEND_POS_SUM|SPEND_OTHER_SUM|TOTAL_DRAWINGS_SPEND|TOTAL_PAYMENTS    |CHANNEL_ATM_RATIO|CHANNEL_POS_RATIO|CHANNEL_OTHER_RATIO|PAYMENT_FIDELITY_RATIO|
+------------+-------------+-------------+---------------+--------------------+------------------+-----------------+-----------------+-------------------+----------------------+
|ES182216044T|0.0          |2.05         |0.0            |2.05                |1241.95           |0.0              |0.6721           |0.0                |407.1967              |
|ES182423065Q|10.8         |0.0          |0.0            |10.8                |3135.15           |0.9153           |0.0              |0.0                |265.6907              |
|ES182215735T|27.0    

2. Diversidad de Interacción

Se calcula un Score de Diversidad contando cuántos tipos de canales de gasto (ATM, POS, OTHER) el cliente ha utilizado al menos una vez.

In [57]:
# --- 2.1. Diversidad de Canales (Repetición/Uso) ---
# Contar los tipos de transacciones con gasto en la ventana
beh_diversity = beh.groupBy("CLIENT_ID").agg(
    # Contamos el número de transacciones con gasto en cada canal
    F.sum(F.when(F.col("CREDIT_CARD_DRAWINGS_ATM") > 0, 1).otherwise(0)).alias("NUM_ATM_TXN"),
    F.sum(F.when(F.col("CREDIT_CARD_DRAWINGS_POS") > 0, 1).otherwise(0)).alias("NUM_POS_TXN"),
    F.sum(F.when(F.col("CREDIT_CARD_DRAWINGS_OTHER") > 0, 1).otherwise(0)).alias("NUM_OTHER_TXN")
).withColumn(
    "CHANNEL_DIVERSITY_SCORE",
    # CHANNEL_DIVERSITY_SCORE: Suma 1 punto por cada tipo de canal que tenga al menos una transacción (TXN > 0).
    F.when(F.col("NUM_ATM_TXN") > 0, 1).otherwise(0) +
    F.when(F.col("NUM_POS_TXN") > 0, 1).otherwise(0) +
    F.when(F.col("NUM_OTHER_TXN") > 0, 1).otherwise(0)
).drop("NUM_ATM_TXN", "NUM_POS_TXN", "NUM_OTHER_TXN") # Mantenemos el score final


# Unimos el score al DataFrame de métricas de interacción
interaccion_df = interaccion_df.join(beh_diversity, "CLIENT_ID", "left")

print("\n================ 2. DIVERSIDAD DE CANALES (REPETICIÓN) ================\n")
print("Score: 1 punto por cada tipo de gasto de tarjeta utilizado (ATM, POS, Other).")
interaccion_df.groupBy("CHANNEL_DIVERSITY_SCORE").count().orderBy("CHANNEL_DIVERSITY_SCORE").show()


================ 2. DIVERSIDAD DE CANALES (REPETICIÓN) ================

Score: 1 punto por cada tipo de gasto de tarjeta utilizado (ATM, POS, Other).
+-----------------------+-----+
|CHANNEL_DIVERSITY_SCORE|count|
+-----------------------+-----+
|                      0|14652|
|                      1|15582|
|                      2|14730|
|                      3| 1082|
+-----------------------+-----+



3. Segmentación de Interacción (I-Score)

Se combinan el Ratio de Fidelidad y el Score de Diversidad para crear una segmentación de tres niveles (Alta, Media, Baja).

In [58]:
# ------------------------------------------------------------
# 3. SEGMENTACIÓN DE INTERACCIÓN (I-SCORE)
# ------------------------------------------------------------

# I-Score: Mide la calidad (Fidelidad) y la amplitud (Diversidad) de la interacción.

# Definir umbrales de Fidelidad usando percentiles (33% y 66%).
fidelidad_quantiles = interaccion_df.approxQuantile("PAYMENT_FIDELITY_RATIO", [0.33, 0.66], 0.01)
f_p33, f_p66 = fidelidad_quantiles

# Umbral de Diversidad: Usar al menos 2 canales (score >= 2) se considera Alta Diversidad.
diversidad_umbral = 2

interaccion_df = interaccion_df.withColumn(
    "INTERACTION_SEGMENT",
    # Segmento ALTA: Cumple con alta fidelidad (percentil 66) Y alta diversidad (2 o 3 canales).
    F.when((F.col("PAYMENT_FIDELITY_RATIO") >= f_p66) & (F.col("CHANNEL_DIVERSITY_SCORE") >= diversidad_umbral), "Alta")
     # Segmento MEDIA: Cumple con media fidelidad (percentil 33) O ha usado al menos 1 canal.
     .when((F.col("PAYMENT_FIDELITY_RATIO") >= f_p33) | (F.col("CHANNEL_DIVERSITY_SCORE") >= 1), "Media")
     # Segmento BAJA: Resto de clientes que tienen baja fidelidad y/o nula diversidad de canales.
     .otherwise("Baja") 
)

print("\n================ 3. SEGMENTACIÓN FINAL DE INTERACCIÓN (I-SCORE) ================\n")
interaccion_df.groupBy("INTERACTION_SEGMENT").count().show()


================ 3. SEGMENTACIÓN FINAL DE INTERACCIÓN (I-SCORE) ================

+-------------------+-----+
|INTERACTION_SEGMENT|count|
+-------------------+-----+
|               Alta| 5620|
|              Media|26900|
|               Baja|13526|
+-------------------+-----+



4. Unión al DataFrame Master

In [59]:
# ------------------------------------------------------------
# 4. UNIÓN AL MASTER
# ------------------------------------------------------------

# Seleccionamos solo las columnas de métricas finales para la unión, descartando auxiliares.
cols_to_keep = ["CLIENT_ID", "CHANNEL_ATM_RATIO", "CHANNEL_POS_RATIO", "CHANNEL_OTHER_RATIO",
                "PAYMENT_FIDELITY_RATIO", "CHANNEL_DIVERSITY_SCORE", "INTERACTION_SEGMENT"]

interaccion_df_final = interaccion_df.select(*cols_to_keep)

# Unimos las métricas al DataFrame maestro de clientes (`df_master`) mediante el CLIENT_ID.
df_master = df_master.join(interaccion_df_final, on="CLIENT_ID", how="left")

print("\n================ 4. UNIÓN AL MASTER ================\n")
print("Métricas de interacción y fidelidad añadidas al master correctamente.")
print(f"Nuevo total de columnas: {len(df_master.columns)}\n")


================ 4. UNIÓN AL MASTER ================

Métricas de interacción y fidelidad añadidas al master correctamente.
Nuevo total de columnas: 64



### RIESGO POTENCIAL

In [ ]:
from datetime import date, timedelta

# ==========================================
# 0. PREPARACIÓN DE FECHAS CLAVE (SOLUCIÓN ESTABLE)
# ==========================================

# 1. Definir la fecha de análisis
ANALYSIS_DATE = date.today()
ANALYSIS_DATE_LIT = F.lit(ANALYSIS_DATE)

# 2. CONVERSIÓN Y RENOMBRE DE LA COLUMNA DE FECHA
# Objetivo: Garantizar que la columna de fecha sea un tipo de dato DateType y tenga un nombre único ('TXN_DATE')
# para evitar conflictos con el String original ('DATE').

# A. Convertir la columna 'DATE' a DateType y guardarla en una columna temporal
beh = beh.withColumn(
    "TXN_DATE_TEMP", 
    F.to_date(F.col("DATE"), "yyyy-MM-dd") # La conversión ES OBLIGATORIA para F.max y F.datediff.
)

# B. ELIMINAR la columna original 'DATE' para evitar la ambigüedad que causó el error
if "DATE" in beh.columns:
    beh = beh.drop("DATE")

# C. Renombrar la columna temporal a 'TXN_DATE'
beh = beh.withColumnRenamed("TXN_DATE_TEMP", "TXN_DATE")


# ------------------------------------------------------------
# 1. MÉTRICAS DE RIESGO POTENCIAL (RECENCIA Y DURACIÓN)
# ------------------------------------------------------------

riesgo_df = beh.groupBy("CLIENT_ID").agg(
    # Recencia: Fecha de la última transacción (gasto o pago). Ahora usamos la columna estable 'TXN_DATE'.
    F.max("TXN_DATE").alias("LAST_TXN_DATE"), 

    # Frecuencia: Número total de transacciones.
    F.count("*").alias("TOTAL_TXN_COUNT"),

    # Frecuencia de Gasto: Número de veces que el cliente tuvo un gasto > 0.
    F.sum(F.when(F.col("KPI_TOTAL_SPEND") > 0, 1).otherwise(0)).alias("SPEND_TXN_COUNT")
)


# --- 1.1. Cálculo de Recencia, Inactividad y Frecuencia ---

riesgo_df = riesgo_df.withColumn(
    # 1. Recencia (Days since last activity): Días entre la fecha de análisis y la última transacción.
    "DAYS_SINCE_LAST_TXN",
    F.datediff(ANALYSIS_DATE_LIT, F.col("LAST_TXN_DATE"))
).withColumn(
    # 2. Inactividad (Days without spend).
    "INACTIVITY_DAYS",
    F.col("DAYS_SINCE_LAST_TXN")
).withColumn(
    # 3. Ratio de Frecuencia de Gasto.
    "SPEND_FREQUENCY_RATIO",
    F.round(F.col("SPEND_TXN_COUNT") / F.col("TOTAL_TXN_COUNT"), 4)
)


print("\n================ 1. INDICADORES DE RECENCIA Y ACTIVIDAD ================\n")
print("Recencia (días desde última TXN) y Frecuencia de Gasto:")
riesgo_df.select("DAYS_SINCE_LAST_TXN", "SPEND_FREQUENCY_RATIO").describe().show()

NameError: name 'F' is not defined

In [ ]:
# ------------------------------------------------------------
# 2. SEGMENTACIÓN DE RIESGO (BASADO EN INACTIVIDAD)
# ------------------------------------------------------------

# Definimos umbrales comunes de inactividad:
INACTIVE_THRESHOLD = 30
CHURN_THRESHOLD = 90

riesgo_df = riesgo_df.withColumn(
    "RISK_SEGMENT",
    F.when(F.col("INACTIVITY_DAYS") >= CHURN_THRESHOLD, "Alto")
     .when(F.col("INACTIVITY_DAYS") >= INACTIVE_THRESHOLD, "Medio")
     .otherwise("Bajo")
)

print("\n================ 2. SEGMENTACIÓN DE RIESGO DE ABANDONO ================\n")
riesgo_df.groupBy("RISK_SEGMENT").count().show()


In [ ]:
# ------------------------------------------------------------
# 3. UNIÓN AL MASTER
# ------------------------------------------------------------

cols_to_keep_risk = [
    "CLIENT_ID",
    "LAST_TXN_DATE",
    "DAYS_SINCE_LAST_TXN",
    "TOTAL_TXN_COUNT",
    "SPEND_FREQUENCY_RATIO",
    "RISK_SEGMENT"
]

riesgo_df_final = riesgo_df.select(*cols_to_keep_risk)

df_master = df_master.join(riesgo_df_final, on="CLIENT_ID", how="left")

print("\n================ 3. UNIÓN AL MASTER ================\n")
print("Métricas de riesgo potencial añadidas al master correctamente.")
print(f"Nuevo total de columnas: {len(df_master.columns)}\n")

NameError: name 'riesgo_df' is not defined

### OPORTUNIDADES COMERCIALES

### ANÁLISIS DE CAUSALIDAD / UPLIFT (para identificar qué ofertas realmente causan más retención)

### EMBEDDING DE COMPORTAMIENTO - SEQUENCE MODELING - (para recomendar productos RNNs / transformers si hay consecuencias largas)

### ANOMALÍA TRANSACCIONAL (para detectar fraudes o glitches del sistema)